# Principal Component Analysis (PCA) and Multiple Factor Analysis (MFA) Tutorial

This notebook explores dimensionality reduction techniques using tumor diagnosis data. We'll cover the theoretical concepts and practical implementation of both PCA and MFA.

## Contents
1. [Data Loading and Exploration](#data)
2. [Principal Component Analysis (PCA)](#pca)
   - Theory and Concepts
   - Implementation
   - Visualization and Interpretation
3. [Multiple Factor Analysis (MFA)](#mfa)
   - Theory and Concepts
   - Implementation
   - Visualization and Interpretation
4. [Comparison and Conclusions](#conclusions)

## 1. Data Loading and Exploration
Let's start by loading the tumor diagnosis dataset and exploring its structure.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the tumor diagnosis data
df = pd.read_excel('tumour_diagnosis.xlsx')
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Explore the dataset structure
print("Column names:")
print(df.columns.tolist())
print(f"Number of features: {len(df.columns)-1}")  # Assuming one column is the target
print("Data types:")
print(df.dtypes.value_counts())
print("Missing values:")
print(df.isnull().sum().sum())
print("Basic statistics:")
df.describe()

## 2. Principal Component Analysis (PCA)

### Theory and Concepts

Principal Component Analysis (PCA) is a dimensionality reduction technique that transforms high-dimensional data into a lower-dimensional space while preserving as much variance as possible.

#### Key Concepts:

**1. Principal Components**: These are new variables that are linear combinations of the original features. They are ordered by the amount of variance they explain in the data.

**2. Eigenvalues and Eigenvectors**: 
- **Eigenvectors** determine the direction of the principal components
- **Eigenvalues** represent the amount of variance explained by each component

**3. Variance Explained**: Each principal component explains a certain percentage of the total variance in the dataset.

#### Mathematical Foundation:

For a data matrix X (n × p), PCA finds directions (principal components) along which the data varies the most:

1. **Standardize the data**: X_standardized = (X - μ) / σ
2. **Compute covariance matrix**: C = (1/(n-1)) * X^T * X
3. **Find eigenvalues and eigenvectors**: C * v = λ * v
4. **Transform data**: Y = X * V (where V is the matrix of eigenvectors)

#### When to use PCA:
- High-dimensional data with potential redundancy
- Visualization of complex datasets
- Noise reduction
- Feature extraction for machine learning
- Data compression

### PCA Implementation

Let's prepare our data and apply PCA to the tumor diagnosis dataset.

In [ ]:
# Prepare data for PCA
# Identify the target column (usually the first or last column for diagnosis data)
# Let's assume the first column is ID and second is diagnosis
if 'diagnosis' in df.columns:
    target_col = 'diagnosis'
elif 'Diagnosis' in df.columns:
    target_col = 'Diagnosis'
else:
    # Take the second column as target if no diagnosis column found
    target_col = df.columns[1]

print(f"Target column: {target_col}")
print(f"Target values: {df[target_col].unique()}")

# Separate features and target, excluding ID columns
# Get numeric columns and remove target and ID columns
X = df.select_dtypes(include=[np.number])
columns_to_exclude = [target_col]

# Identify and exclude ID columns (common patterns)
id_columns = [col for col in X.columns if 'id' in col.lower() or 'ID' in col or col.lower().startswith('id_')]
columns_to_exclude.extend(id_columns)

# Remove excluded columns
X = X.drop(columns=columns_to_exclude, errors='ignore')

# Get target variable
y = df[target_col] if target_col in df.columns else None

print(f"\nExcluded columns: {columns_to_exclude}")
print(f"Feature matrix shape: {X.shape}")
print(f"Features: {X.columns.tolist()[:5]}...")  # Show first 5 features

In [ ]:
# Standardize the features (important for PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Data standardized. Mean should be ~0, std should be ~1:")
print(f"Mean: {np.mean(X_scaled, axis=0)[:5]}")
print(f"Std: {np.std(X_scaled, axis=0)[:5]}")

In [ ]:
# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

print(f"Original data shape: {X.shape}")
print(f"PCA transformed shape: {X_pca.shape}")
print(f"Explained variance ratio (first 10 components):")
print(pca.explained_variance_ratio_[:10])
print(f"Cumulative explained variance (first 10 components):")
print(np.cumsum(pca.explained_variance_ratio_)[:10])

### PCA Visualization and Interpretation

In [ ]:
# Plot explained variance
plt.figure(figsize=(12, 5))

# Scree plot
plt.subplot(1, 2, 1)
plt.plot(range(1, min(21, len(pca.explained_variance_ratio_)+1)),
    pca.explained_variance_ratio_[:20], 'bo-')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.grid(True, alpha=0.3)

In [ ]:
# 2D PCA visualization
if y is not None:
    plt.figure(figsize=(10, 6))
    
    # Create scatter plot colored by target
    unique_labels = np.unique(y)
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
    
    for i, label in enumerate(unique_labels):
        mask = y == label
        plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                   c=[colors[i]], label=f'{label}', alpha=0.6, s=50)
    
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('PCA: First Two Principal Components')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    # If no target variable, just plot the points
    plt.figure(figsize=(8, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.6)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('PCA: First Two Principal Components')
    plt.grid(True, alpha=0.3)
    plt.show()

### PCA Biplot - Feature Contribution Visualization

A **biplot** is what you're thinking of when you want to see "features as lines" on the PCA plot. It combines:

1. **Observations** (data points) as dots
2. **Feature loadings** (contributions) as arrows/vectors

The arrows show:
- **Direction**: How each feature contributes to PC1 and PC2
- **Length**: The magnitude of contribution 
- **Angle**: Features pointing in similar directions are correlated

**Interpretation**:
- Arrows pointing in the same direction represent positively correlated features
- Arrows pointing in opposite directions represent negatively correlated features  
- Longer arrows indicate features with stronger influence on the principal components
- The angle between an arrow and a PC axis shows how much that feature contributes to that PC

In [ ]:
# PCA Biplot - showing both observations and feature contributions
def create_biplot(X_pca, pca, feature_names, y=None, pc1=0, pc2=1):
    """
    Create a biplot showing both PCA scores and loadings
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot observations (scores)
    if y is not None:
        unique_labels = np.unique(y)
        colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
        
        for i, label in enumerate(unique_labels):
            mask = y == label
            ax.scatter(X_pca[mask, pc1], X_pca[mask, pc2], 
                      c=[colors[i]], label=f'{label}', alpha=0.6, s=50)
    else:
        ax.scatter(X_pca[:, pc1], X_pca[:, pc2], alpha=0.6, s=50)
    
    # Plot loadings (feature contributions) as arrows
    loadings = pca.components_[[pc1, pc2]].T  # Shape: (n_features, 2)
    
    # Scale loadings to fit nicely with the scores
    scale_factor = 0.7 * max(np.abs(X_pca[:, [pc1, pc2]]).max(), np.abs(loadings).max())
    loadings_scaled = loadings * scale_factor
    
    # Only show top contributing features to avoid clutter
    loading_magnitudes = np.sqrt(loadings[:, 0]**2 + loadings[:, 1]**2)
    top_features_idx = np.argsort(loading_magnitudes)[-10:]  # Top 10 features
    
    for i in top_features_idx:
        ax.arrow(0, 0, loadings_scaled[i, 0], loadings_scaled[i, 1],
                head_width=0.05*scale_factor, head_length=0.05*scale_factor,
                fc='red', ec='red', alpha=0.7)
        
        # Add feature labels
        ax.text(loadings_scaled[i, 0]*1.1, loadings_scaled[i, 1]*1.1,
               feature_names[i], fontsize=8, ha='center', va='center',
               bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))
    
    ax.set_xlabel(f'PC{pc1+1} ({pca.explained_variance_ratio_[pc1]:.1%} variance)')
    ax.set_ylabel(f'PC{pc2+1} ({pca.explained_variance_ratio_[pc2]:.1%} variance)')
    ax.set_title('PCA Biplot: Observations and Feature Contributions')
    
    if y is not None:
        ax.legend(loc='upper right')
    
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    
    return fig, ax

# Create the biplot
fig, ax = create_biplot(X_pca, pca, X.columns.tolist(), y)
plt.show()

## 3. Multiple Factor Analysis (MFA)

### Theory and Concepts

Multiple Factor Analysis (MFA) is an extension of Principal Component Analysis designed to handle datasets with multiple groups of variables. It's particularly useful when dealing with mixed-type data or when variables can be naturally grouped.

#### Key Concepts:

**1. Variable Groups**: MFA assumes that variables can be grouped into meaningful categories (e.g., morphological features, texture features, etc.).

**2. Weighted PCA**: Each group of variables is first analyzed separately using PCA, then combined using weights that balance the contribution of each group.

**3. Global Analysis**: After analyzing each group separately, MFA performs a global analysis that takes into account the structure revealed by each group.

#### Mathematical Foundation:

1. **Group-wise PCA**: For each group j, perform PCA and retain the first αⱼ components
2. **Weight Calculation**: Weight each group by wⱼ = 1/λ₁ⱼ (where λ₁ⱼ is the first eigenvalue of group j)
3. **Global Analysis**: Perform PCA on the weighted and concatenated data

#### Advantages of MFA over PCA:
- Handles grouped variables effectively
- Prevents one group from dominating the analysis
- Provides interpretation at both group and global levels
- Better suited for mixed-type data

#### When to use MFA:
- Variables can be naturally grouped
- Some groups have many more variables than others
- You want to understand both group-specific and global patterns
- Working with mixed data types

### MFA Implementation

Since scikit-learn doesn't have a direct MFA implementation, we'll create a simplified version that demonstrates the key concepts. For tumor diagnosis data, we might group features by their types (e.g., mean values, standard errors, worst values).

In [ ]:
# Create feature groups for MFA
# For typical tumor data, features often come in groups (mean, SE, worst)
feature_names = X.columns.tolist()
print("Feature names sample:", feature_names[:10])

# Try to identify groups based on naming patterns
groups = {}
if any('_mean' in col or col.endswith('_m') for col in feature_names):
    groups['mean'] = [col for col in feature_names if '_mean' in col or col.endswith('_m')]
if any('_se' in col or 'se_' in col for col in feature_names):
    groups['se'] = [col for col in feature_names if '_se' in col or 'se_' in col]
if any('_worst' in col or col.endswith('_w') for col in feature_names):
    groups['worst'] = [col for col in feature_names if '_worst' in col or col.endswith('_w')]

# If no clear naming pattern, create groups by feature index
if not groups:
    n_features = len(feature_names)
    group_size = n_features // 3
    groups = {
        'group1': feature_names[:group_size],
        'group2': feature_names[group_size:2*group_size],
        'group3': feature_names[2*group_size:]
    }

print("\nFeature groups:")
for group_name, features in groups.items():
    print(f"{group_name}: {len(features)} features")
    print(f"  Sample features: {features[:3]}")

In [ ]:
# Implement simplified MFA
class SimpleMFA:
    def __init__(self, groups, n_components=2):
        self.groups = groups
        self.n_components = n_components
        self.group_pcas = {}
        self.group_weights = {}
        self.global_pca = None
        
    def fit_transform(self, X):
        # Step 1: Perform PCA on each group
        group_scores = []
        
        for group_name, feature_list in self.groups.items():
            # Get data for this group
            group_data = X[feature_list]
            
            # Standardize group data
            scaler = StandardScaler()
            group_scaled = scaler.fit_transform(group_data)
            
            # Apply PCA to group
            pca = PCA()
            group_pca_scores = pca.fit_transform(group_scaled)
            
            # Store PCA object and calculate weight
            self.group_pcas[group_name] = pca
            self.group_weights[group_name] = 1.0 / pca.explained_variance_[0]  # Weight by first eigenvalue
            
            # Keep all components for now
            group_scores.append(group_pca_scores)
            
            print(f"Group {group_name}:")
            print(f"  First eigenvalue: {pca.explained_variance_[0]:.3f}")
            print(f"  Weight: {self.group_weights[group_name]:.3f}")
            print(f"  Explained variance (first 3 components): {pca.explained_variance_ratio_[:3]}")
        
        # Step 2: Combine weighted group scores
        weighted_scores = []
        for i, (group_name, scores) in enumerate(zip(self.groups.keys(), group_scores)):
            weight = self.group_weights[group_name]
            weighted_scores.append(scores * np.sqrt(weight))  # Apply square root of weight
        
        # Concatenate all weighted scores
        combined_scores = np.hstack(weighted_scores)
        
        # Step 3: Apply global PCA
        self.global_pca = PCA(n_components=self.n_components)
        mfa_scores = self.global_pca.fit_transform(combined_scores)
        
        return mfa_scores

# Apply MFA
mfa = SimpleMFA(groups, n_components=10)
X_mfa = mfa.fit_transform(X)

print(f"MFA Results:")
print(f"MFA transformed shape: {X_mfa.shape}")
print(f"Explained variance ratio: {mfa.global_pca.explained_variance_ratio_}")
print(f"Cumulative explained variance: {np.cumsum(mfa.global_pca.explained_variance_ratio_)}")

## 4. Comparison and Conclusions

Let's compare PCA and MFA results and draw conclusions about their effectiveness for this tumor diagnosis dataset.

In [ ]:
# Compare PCA and MFA visualizations
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

if y is not None:
    unique_labels = np.unique(y)
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
    
    # PCA plot
    for i, label in enumerate(unique_labels):
        mask = y == label
        axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], 
                       c=[colors[i]], label=f'{label}', alpha=0.6, s=50)
    
    axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    axes[0].set_title('PCA: First Two Principal Components')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # MFA plot
    for i, label in enumerate(unique_labels):
        mask = y == label
        axes[1].scatter(X_mfa[mask, 0], X_mfa[mask, 1], 
                       c=[colors[i]], label=f'{label}', alpha=0.6, s=50)
    
    axes[1].set_xlabel(f'MFA1 ({mfa.global_pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[1].set_ylabel(f'MFA2 ({mfa.global_pca.explained_variance_ratio_[1]:.1%} variance)')
    axes[1].set_title('MFA: First Two Components')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
else:
    # No target variable
    axes[0].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.6)
    axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    axes[0].set_title('PCA: First Two Principal Components')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].scatter(X_mfa[:, 0], X_mfa[:, 1], alpha=0.6)
    axes[1].set_xlabel(f'MFA1 ({mfa.global_pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[1].set_ylabel(f'MFA2 ({mfa.global_pca.explained_variance_ratio_[1]:.1%} variance)')
    axes[1].set_title('MFA: First Two Components')
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Quantitative comparison
print("=== PCA vs MFA Comparison ===")
print(f"Variance explained by first 2 components:")
print(f"PCA: {np.sum(pca.explained_variance_ratio_[:2]):.3f} ({pca.explained_variance_ratio_[0]:.3f} + {pca.explained_variance_ratio_[1]:.3f})")
print(f"MFA: {np.sum(mfa.global_pca.explained_variance_ratio_[:2]):.3f} ({mfa.global_pca.explained_variance_ratio_[0]:.3f} + {mfa.global_pca.explained_variance_ratio_[1]:.3f})")

print(f"\nComponents needed for 80% variance:")
pca_80 = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.8) + 1
mfa_80 = np.argmax(np.cumsum(mfa.global_pca.explained_variance_ratio_) >= 0.8) + 1
print(f"PCA: {pca_80} components")
print(f"MFA: {mfa_80} components")

print(f"Components needed for 95% variance:")
pca_95 = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95) + 1
mfa_95 = np.argmax(np.cumsum(mfa.global_pca.explained_variance_ratio_) >= 0.95) + 1
print(f"PCA: {pca_95} components")
print(f"MFA: {mfa_95} components")

### Key Findings and Conclusions

**PCA Results:**
- Traditional PCA provides a straightforward dimensionality reduction
- All features are treated equally in the analysis
- Effective for general-purpose dimensionality reduction
- Good baseline for comparison

**MFA Results:**
- Takes into account the grouped structure of features
- Balances contribution from different feature groups
- May provide better interpretation when features have natural groupings
- More complex but potentially more informative for structured data

**When to Choose Each Method:**

**Use PCA when:**
- Features don't have natural groupings
- You want a simple, straightforward analysis
- All features are of similar importance
- You need a quick exploratory analysis

**Use MFA when:**
- Features can be logically grouped
- Some feature groups have many more variables than others
- You want to prevent one group from dominating the analysis
- You're working with mixed-type data or structured datasets

**Practical Recommendations:**
1. Always start with exploratory data analysis to understand your feature structure
2. Consider the interpretability requirements of your analysis
3. Validate results with domain knowledge
4. Consider the computational complexity vs. benefit trade-off
5. Use cross-validation or other methods to assess the stability of your results

Both PCA and MFA are powerful techniques for dimensionality reduction and data visualization. The choice between them depends on your specific data structure and analysis goals.